# Lab 4


## Họ và tên thí sinh : Phạm Thế Hiển
## MSSV : 19529741

# Câu 1 (3 điểm): 

Cho một RDD trong Spark chứa danh sách lớp như cell dưới đây

(a)	(1 điểm) Sử dụng Pyspark, liệt kê các tên trong danh sách trên (mỗi tên chỉ xuất hiện một lần).

(b)	(2 điểm) Sử dụng hàm map() và reduceByKey() (Pyspark), tìm số sinh viên tương ứng với mỗi tên trong danh sách từ Câu (1a).


In [0]:
list_class = ['Tuan', 'Hoang', 'Minh', 'Nga', 'Tam', 'Tuan', 'Minh', 'Tam', 'Tuan', 'Phuong']
rdd_class = sc.parallelize(list_class)

In [0]:
#Sử dụng Pyspark, liệt kê các tên trong danh sách trên (mỗi tên chỉ xuất hiện một lần)
ten_duy_nhat = rdd_class.distinct()
ket_qua_a = ten_duy_nhat.collect()
print(ket_qua_a)


['Minh', 'Nga', 'Tuan', 'Hoang', 'Tam', 'Phuong']


In [0]:
cap_ten_va_so_sinh_vien = rdd_class.map(lambda ten: (ten, 1))
so_luong_sinh_vien_theo_ten = cap_ten_va_so_sinh_vien.reduceByKey(lambda x, y: x + y)
ket_qua_b = so_luong_sinh_vien_theo_ten.collect()
print(ket_qua_b)


[('Minh', 2), ('Nga', 1), ('Tuan', 3), ('Hoang', 1), ('Tam', 2), ('Phuong', 1)]


## Câu 2 (7 điểm)

Trong câu này, bạn sẽ upload file data shakespeare.txt từ thư mục LMS và đếm từ trong file này.

### (2a) (1 điểm) 

Upload file lên LMS và gán vào RDD có tên là **rdd_shp**

In [0]:
import os.path

# Ví dụ
# fileName = "dbfs:/FileStore/shared_uploads/nguyenchikien@iuh.edu.vn/shakespeare.txt"

# Your code goes here
fileName = "dbfs:/FileStore/shared_uploads/hienchipham1212@gmail.com/shakespeare.txt"
rdd_shp = sc.textFile(fileName)

In [0]:
assert rdd_shp.take(3)==['The Complete Works of William Shakespeare', '', 'by William Shakespeare'], "Assertion Error"

### (2b) (1 điểm)

Viết hàm **removePunctuation** để chuyển một đoạn văn bản thành chữ thường (lower case) và xoá tất cả các dấu chấm câu. Áp dụng hàm này cho **rdd_shp** để nhận được RDD **rdd_shp_2**

In [0]:
import re
def removePunctuation(text):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        text (str): A string.

    Returns:
        str: The cleaned up string.
    """
    
    # Your code goes here
    text_lower = text.lower()
    text_no_punct = re.sub(r'[^a-zA-Z0-9 ]', '', text_lower)
    cleaned_text = text_no_punct.strip()
    return cleaned_text
    

In [0]:
# Áp dụng hàm này cho rdd_shp để nhận được RDD rdd_shp_2

# Your code goes here
rdd_shp_2 = rdd_shp.map(removePunctuation)
rdd_shp_2.take(20)

Out[32]: ['the complete works of william shakespeare',
 '',
 'by william shakespeare',
 '',
 '',
 '',
 '',
 'contents',
 '',
 'the sonnets',
 'alls well that ends well',
 'the tragedy of antony and cleopatra',
 'as you like it',
 'the comedy of errors',
 'the tragedy of coriolanus',
 'cymbeline',
 'the tragedy of hamlet prince of denmark',
 'the first part of king henry the fourth',
 'the second part of king henry the fourth',
 'the life of king henry the fifth']

### (2c) 2 điểm

Tạo RDD **rdd_words** gồm các words từ **rdd_shp_2** bằng cách split **rdd_shp_2** sử dụng dấu cách (space). In ra 5 từ đầu tiên của **rdd_words** và số lượng từ trong **rdd_words**.

In [0]:
# Your code goes here
rdd_words = rdd_shp_2.flatMap(lambda line: line.split(" "))
print("5 từ đầu tiên của rdd_words:")
print(rdd_words.take(5))
word_count = rdd_words.count()
print(f"Số lượng từ trong rdd_words: {word_count}")


5 từ đầu tiên của rdd_words:
['the', 'complete', 'works', 'of', 'william']
Số lượng từ trong rdd_words: 1002968


### (2d) 1 điểm

Tạo RDD **rdd_words_2** từ **rdd_words** bằng cách xoá tất cả các empty word (''). In ra số lượng từ trong **rdd_words_2**.

In [0]:
rdd_words_2 = rdd_words.filter(lambda word: word != '')
word_count_2 = rdd_words_2.count()
print(f"Số lượng từ trong rdd_words_2: {word_count_2}")


Số lượng từ trong rdd_words_2: 958665


### (2e) 2 điểm

Sử dụng code từ câu (1b), viết một function nhận một RDD với một danh sách word đầu vào và đếm số lần xuất hiện của mỗi từ. Sử dụng function này để đếm từ trong **rdd_words_2**. **Bravo! Bạn đã hoàn thành việc đếm số lần xuất hiện của tất cả các từ trong toàn bộ các tác phẩm của Shakespeare!** In ra 10 từ được sử dụng nhiều nhất trong toàn bộ các tác phẩm của Shakespeare.

In [0]:
def wordCount(wordListRDD):
    """Creates a pair RDD with word counts from an RDD of words.

    Args:
        wordListRDD (RDD of str): An RDD consisting of words.

    Returns:
        RDD of (str, int): An RDD consisting of (word, count) tuples.
    """
    
    # Your code goes here
    word_count_pairs = wordListRDD.map(lambda word: (word, 1))
    word_counts = word_count_pairs.reduceByKey(lambda x, y: x + y)
    
    return word_counts
rdd_word_counts = wordCount(rdd_words_2)

In [0]:
# In ra 10 từ được sử dụng nhiều nhất trong toàn bộ các tác phẩm của Shakespeare

# Your code goes here
top_words = rdd_word_counts.takeOrdered(10, key=lambda x: -x[1])
print("Top 10 từ được sử dụng nhiều nhất:")
for word, count in top_words:
    print(f"{word}: {count}")

Top 10 từ được sử dụng nhiều nhất:
the: 30056
and: 28329
i: 21970
to: 20854
of: 18689
a: 16157
you: 14367
my: 13183
in: 12202
that: 11762
